In [61]:
# run scraper.py to scrap data and save it to review_text_all.txt
import scraper as s
s.runScraper()

In [1]:
import itertools as it
import os
import pandas as pd
import spacy

In [2]:
nlp = spacy.load('en')

## Phrase Modeling

Phrase modeling is an approach to learning combinations of tokens that together represent meaning multi-word concepts.  These phrase models are developed by looping over the words in the corpus and finding words that appear together more than they should by random chance.  The formula used to determine if whether two tokens $A$ and $B$ constitute a phrase is:

$$
\frac {count(A B) - count_{min}}
{count(A) * count(B)}
*N > threshold
$$

where:
* $count(A)$ is the number of times $A$ appears in the corpus
* $count(B)$ is the number of times $B$ appears in the corpus
* $count(AB)$ is the number of times $AB$ appear in the corpus in that order
* $N$ is the total size of the corpus vocabulary
* $count_{min}$ is a user-defined parameter to ensure that the phrase appears a minimum number of times
* $threshold$ is a user-defined paramter to control how strong the relationship must be before the two tokens are considered a single concept

Once we have trained the phrase model we can apply it to the reviews in our corpus.  It will consider the multiworded tokens to be single phrases.

The gensim library will help us with phrase modeling, specifically the Phrases class.

In [3]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models.word2vec import LineSentence

In [4]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [5]:
intermediate_directory = os.path.join('.', 'intermediate')
review_txt_filepath = os.path.join(intermediate_directory,'review_text_all.txt')
ngram_all = os.path.join(intermediate_directory, 'ngram_all')
unigram_sentences_filepath = os.path.join(ngram_all,
                                          'unigram_sentences_all.txt')

We will use `lemmatized_sentence_corpus` generator to loop over the original review text, segmenting the reviews into individual sentences and normalizing the text.  We will write this data back out to a new file (`unigram_sentence_all`), with one normalized sentence per line.  We will use this data for learning our phrase models.

In [6]:
with open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        f.write(sentence + '\n')

Our data in the `unigram_sentences_all` file is now organized as a large text file with one sentence per line.  This formate allows us to use gensim's LineSentence class, a convenient iterator for working with gensim's other components.  It *streams* the documents/sentences from disk, so you never have ot hold the entire corpus in RAM at once.  This allows you to scale the modeling to a very large corpora.

In [7]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [8]:
for unigram_sentence in it.islice(unigram_sentences, 5,7):
    print(u' '.join(unigram_sentence))
    print(u' ')

taste maple vanilla bourbon and oak
 
everything -PRON- want in a beer
 


In [36]:
bigram_model_file = os.path.join(ngram_all, 'bigram_model')

In [37]:
if 1==1:
    phrases = Phrases(unigram_sentences)
    bigram_model = Phraser(phrases)
    bigram_model.save(bigram_model_file)
    
bigram_model = Phrases.load(bigram_model_file)

In [38]:
bigram_sentences_filepath = os.path.join(ngram_all,'bigram_sentences_all.txt')

In [39]:
with open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for unigram_sentence in unigram_sentences:

        bigram_sentence = u' '.join(bigram_model[unigram_sentence])

        f.write(bigram_sentence + '\n')

In [40]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [42]:
for bigram_sentence in it.islice(bigram_sentences, 5, 7):
    print(u' '.join(bigram_sentence))
    print(u'')

taste maple vanilla bourbon and oak

everything -PRON- want in a beer



In [44]:
trigram_model_filepath = os.path.join(ngram_all, 'trigram_model')

In [45]:
if 1 == 1:

    phrases_trigram = Phrases(bigram_sentences)
    trigram_model = Phraser(phrases_trigram)
    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

In [ ]:
if 1==1:
    phrases = Phrases(unigram_sentences)
    bigram_model = Phraser(phrases)
    bigram_model.save(bigram_model_file)
    
bigram_model = Phrases.load(bigram_model_file)

In [46]:
trigram_sentences_filepath = os.path.join(ngram_all, 'trigram_sentences_all.txt')

In [49]:
with open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for bigram_sentence in bigram_sentences:

        trigram_sentence = u' '.join(trigram_model[bigram_sentence])

        f.write(trigram_sentence + '\n')

In [51]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [52]:
for trigram_sentence in it.islice(trigram_sentences, 230, 240):
    print(u' '.join(trigram_sentence))
    print(u'')

a_bit of heat linger as well

good feel

overall though -PRON- do_not claim to be this could very well be barrel aged morning_delight

absolutely fantastic

this have everything -PRON- love about md and add a touch of whisky barrel age on top of -PRON-

one of the rare rare occasion when a beer of this magnitude surpass -PRON- billing

this be the good barrel age imperial stout out there at this point and -PRON- can not imagine be able to surpass this either

would love to drink some of the second batch that be apparently wait patiently at the brewery

thank again cparl

world class



In [53]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')

In [55]:
from spacy.lang.en.stop_words import STOP_WORDS

In [58]:

with open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:

    for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                  batch_size=10000, n_threads=4):

        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review
                          if not punct_space(token)]

        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review
                          if term not in STOP_WORDS]
        
        # remove pronouns
        trigram_review = [term for term in trigram_review
                          if term !='-PRON-']
        

        # write the transformed review as a line in the new file
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

In [59]:
print(u'Original:' + u'\n')

for review in it.islice(line_review(review_txt_filepath), 11, 12):
    print(review)

print(u'----' + u'\n')
print(u'Transformed:' + u'\n')

with open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 11, 12):
        print(review)

Original:

OK, time to drown this puppy. Picked up at the release (where there were numerous Minnesota beer geeks in attendance)  this is the lone  single bottle resting in the fridge. Now resting in my glass. Well, half in my glass, half in my wife's glass. 12 oz. bottle with silver wax.   The pour is dark night, carbonation close to the non-existent side. The nose - wow. Like the best pancake house in the world, with grade A maple syrup made in house  a cold press coffee place beside the kitchen. Then the waitress sets a glass of bourbon next to the syrup. Seriously. Magnificent.   Kentucky Brunch is one of those beers that can't possibly live up to the hype, right? Wrong. More maple syrup taste than any beer I've ever come into contact with -  I like small-batch, artisanal maple syrup almost as much as I like beer. And I swear I can taste the waffles. Big bourbon wallop is cut by just the right amount of coffee; not enough to tip this into an annoying coffee bomb, but enough to get 